# Visualize Results: Downstream Performance - Multiclass Classification Fully Observed Experiments

This notebook should answer the questions: *Does imputation lead to better downstream performances?*

Data needs to be preprocessed with other notebook, her we only import two csv files with raw data regarding the results of the experiment and information about the used datasets!

## Notebook Structure 

* Application Scenario 2 - Downstream Performance  
   * Categorical  Columns (Classification)
   * Numerical Columns (Regression)
   * Heterogenous Columns (Classification and Regression Combined)

In [139]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import re
import seaborn as sns

from pathlib import Path

import plotly as py
import plotly.express as px
import plotly.graph_objects as go
import xarray as xr


%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Settings

In [140]:
sns.set(style="whitegrid")
sns.set_context('paper', font_scale=1.5)
mpl.rcParams['lines.linewidth'] = '2'

In [141]:
EXPERIMENT = "binary_experiment"

#EXPERIMENT_PATH = Path(f"../results/{EXPERIMENT}/")

CLF_METRIC = "Classification Tasks"
REG_METRIC = "Regression Tasks"

DOWNSTREAM_RESULT_TYPE = "downstream_performance_mean"
IMPUTE_RESULT_TYPE = "impute_performance_mean"

FIGURES_PATH = Path(f"../paper/figures/")

## Data Preparation

In [142]:
#read results.csv file here!

# Pick whether you want to analyze the "Regression" Experiment oder the "Regression Corrupted" Experiment

#results = pd.read_csv('regression_corrupted.csv')
results = pd.read_csv('multiclass_classification.csv')
# Preresults.head()

In [143]:
# Filtering the relevant data for downstream analysis

na_impute_results = results[
    (results["result_type"] == IMPUTE_RESULT_TYPE) & 
    (results["metric"].isin(["F1_macro", "RMSE"]))
]
na_impute_results.drop(["baseline", "corrupted", "imputed"], axis=1, inplace=True)
na_impute_results = na_impute_results[na_impute_results.isna().any(axis=1)]
na_impute_results.shape

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\3401284833.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(93, 11)

In [144]:
# check if strategy type is correct!
STRATEGY_TYPE = "single_single"

downstream_results = results[
    (results["result_type"] == DOWNSTREAM_RESULT_TYPE) & 
    (results["metric"].isin(["F1_macro", "RMSE"]) &
    (results["strategy"] == STRATEGY_TYPE))
]

# remove experiments where imputation failed
downstream_results = downstream_results.merge(
    na_impute_results,
    how = "left",
    validate = "one_to_one",
    indicator = True,
    suffixes=("", "_imp"),
    on = ["experiment", "imputer", "task", "missing_type", "missing_fraction", "strategy", "column"]
)
downstream_results = downstream_results[downstream_results["_merge"]=="left_only"]

assert len(results["strategy"].unique()) == 2
downstream_results.drop(["experiment", "strategy", "result_type_imp", "metric_imp", "train", "test", "train_imp", "test_imp", "_merge"], axis=1, inplace=True)

downstream_results = downstream_results.rename(
    {
        "imputer": "Imputation_Method",
        "task": "Task",
        "missing_type": "Missing Type",
        "missing_fraction": "Missing Fraction",
        "column": "Column",
        "baseline": "Baseline",
        "imputed": "Imputed",
        "corrupted": "Corrupted"
    },
    axis = 1
)

In [145]:
rename_imputer_dict = {
    "ModeImputer": "Mean/Mode",
    "KNNImputer": "KNN",
    "ForestImputer": "Random Forest",
    "AutoKerasImputer": "Discriminative DL",
    "VAEImputer": "VAE",
    "GAINImputer": "GAIN"    
}

rename_metric_dict = {
    "F1_macro": CLF_METRIC,
    "RMSE": REG_METRIC
}

downstream_results = downstream_results.replace(rename_imputer_dict)
downstream_results = downstream_results.replace(rename_metric_dict)

downstream_results

,Imputation_Method,Task,Missing Type,Missing Fraction,Column,result_type,metric,Baseline,Corrupted,Imputed
0,Discriminative DL,1459,MAR,0.01,V7,downstream_performance_mean,Classification Tasks,0.316824,0.314698,0.317027
1,Discriminative DL,1459,MAR,0.10,V7,downstream_performance_mean,Classification Tasks,0.316824,0.283957,0.313391
2,Discriminative DL,1459,MAR,0.30,V7,downstream_performance_mean,Classification Tasks,0.316824,0.235809,0.307565
3,Discriminative DL,1459,MAR,0.50,V7,downstream_performance_mean,Classification Tasks,0.316824,0.195967,0.322023
4,Discriminative DL,1459,MCAR,0.01,V7,downstream_performance_mean,Classification Tasks,0.316824,0.313686,0.317737
...,...,...,...,...,...,...,...,...,...,...
1218,VAE,6,MCAR,0.50,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.671203,0.713386
1219,VAE,6,MNAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.721082,0.721990
1220,VAE,6,MNAR,0.10,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.707826,0.721241
1221,VAE,6,MNAR,0.30,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.692779,0.718277


### Robustness: check which imputers yielded `NaN`values

In [146]:
for col in downstream_results.columns:
    na_sum = downstream_results[col].isna().sum()
    if na_sum > 0:
        print("-----" * 10)        
        print(col, na_sum)
        print("-----" * 10)        
        na_idx = downstream_results[col].isna()
        print(downstream_results.loc[na_idx, "Imputation Method"].value_counts(dropna=False))
        print("\n")

## Compute Downstream Performance relative to Baseline

In [147]:
clf_row_idx = downstream_results["metric"] == CLF_METRIC
reg_row_idx = downstream_results["metric"] == REG_METRIC

In [148]:
downstream_results["Improvement"]   = (downstream_results["Imputed"] - downstream_results["Corrupted"]  ) / downstream_results["Baseline"]
downstream_results.loc[reg_row_idx, "Improvement"]   = downstream_results.loc[reg_row_idx, "Improvement"]   * -1

#mar001.drop(["Missing Type", "Missing Fraction", "Column", "result_type", "metric", "Baseline", "Imputed", "Corrupted", "Unnamed: 0"], axis=1, inplace=True)

#print(downstream_results)
#downstream_results.to_csv('downstream_results.csv')
downstream_results.head()

,Imputation_Method,Task,Missing Type,Missing Fraction,Column,result_type,metric,Baseline,Corrupted,Imputed,Improvement
0,Discriminative DL,1459,MAR,0.01,V7,downstream_performance_mean,Classification Tasks,0.316824,0.314698,0.317027,0.007353
1,Discriminative DL,1459,MAR,0.10,V7,downstream_performance_mean,Classification Tasks,0.316824,0.283957,0.313391,0.092903
2,Discriminative DL,1459,MAR,0.30,V7,downstream_performance_mean,Classification Tasks,0.316824,0.235809,0.307565,0.226487
3,Discriminative DL,1459,MAR,0.50,V7,downstream_performance_mean,Classification Tasks,0.316824,0.195967,0.322023,0.397874
4,Discriminative DL,1459,MCAR,0.01,V7,downstream_performance_mean,Classification Tasks,0.316824,0.313686,0.317737,0.012786


## Adding Dataset Info, Sorting and Ranking

In [149]:
# Sortierung der Daten

#downstream_results_full_sort = pd.read_csv('downstream_results.csv')
downstream_results_full_sort = downstream_results

#df = sns.load_dataset('impute_results_full')
#downstream_results_full_sort = downstream_results_full_sort.replace('$k$-NN','KNN')
#impute_results_full_sort.head()

#impute_results_full_sort = impute_results_full_sort.sort_values(['Task'], ascending=[True])
downstream_results_full_sort = downstream_results_full_sort.sort_values(['Task', 'Missing Type', 'Missing Fraction', 'Improvement'], ascending=[True, True, True, True])
#print(downstream_results_full_sort)
downstream_results_full_sort.head()


#downstream_results_full_sort.to_csv('downstream_results_full_sort.csv')

,Imputation_Method,Task,Missing Type,Missing Fraction,Column,result_type,metric,Baseline,Corrupted,Imputed,Improvement
1211,VAE,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.721942,0.721804,-0.000192
191,Discriminative DL,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.721631,0.722056,0.000588
395,Random Forest,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.721506,0.722144,0.000884
1007,Mean/Mode,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.720902,0.721964,0.001471
803,KNN,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.720294,0.722247,0.002704


In [150]:
# add dataset information from other csv file

dataset_info = pd.read_csv('datasets_information_overview.csv')
dataset_info = dataset_info.rename(columns={"did": "Task"})


downstream_results_full_sort = pd.merge(downstream_results_full_sort, dataset_info, on='Task')
#downstream_results_full_sort.to_csv('downstream_results_full_sort_testtesttest.csv')
downstream_results_full_sort.head()

,Imputation_Method,Task,Missing Type,Missing Fraction,Column,result_type,metric,Baseline,Corrupted,Imputed,Improvement,Unnamed: 0,name,MajorityClassSize,MinorityClassSize,NumberOfFeatures,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures,NumberOfClasses
0,VAE,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.721942,0.721804,-0.000192,59,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0
1,Discriminative DL,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.721631,0.722056,0.000588,59,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0
2,Random Forest,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.721506,0.722144,0.000884,59,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0
3,Mean/Mode,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.720902,0.721964,0.001471,59,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0
4,KNN,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.720294,0.722247,0.002704,59,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0


In [151]:
# Ranking of downstream performance per data constellation

EXPERIMENTAL_CONDITIONS = ["Task", "Missing Type", "Missing Fraction", "Column", "result_type"]

downstream_results_rank = downstream_results_full_sort

#clf_row_idx = impute_results["metric"] == CLF_METRIC
#reg_row_idx = impute_results["metric"] == REG_METRIC

downstream_results_rank["Downstream Performance Rank"] = downstream_results_rank.groupby(EXPERIMENTAL_CONDITIONS).rank(ascending=False, na_option="bottom", method="min")["Improvement"]
downstream_results_rank.to_csv('downstream_results_complete_overview.csv')
downstream_results_rank.head()


,Imputation_Method,Task,Missing Type,Missing Fraction,Column,result_type,metric,Baseline,Corrupted,Imputed,...,Unnamed: 0,name,MajorityClassSize,MinorityClassSize,NumberOfFeatures,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures,NumberOfClasses,Downstream Performance Rank
0,VAE,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.721942,0.721804,...,59,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,5.0
1,Discriminative DL,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.721631,0.722056,...,59,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,4.0
2,Random Forest,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.721506,0.722144,...,59,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,3.0
3,Mean/Mode,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.720902,0.721964,...,59,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,2.0
4,KNN,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.720294,0.722247,...,59,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,1.0


In [152]:
# Merge the two columns "Missing Type" and "Missing Fraction"

downstream_results_rank['Missing Type'] = downstream_results_rank['Missing Type'].astype(str)
downstream_results_rank['Missing Fraction'] = downstream_results_rank['Missing Fraction'].astype(str)
datatype_new = downstream_results_rank.dtypes
#print(datatype_new)

downstream_results_rank['Data_Constellation'] = downstream_results_rank['Missing Type'] + ' - ' + downstream_results_rank['Missing Fraction']
downstream_results_rank.to_csv('downstream_results_rank_temp.csv')
downstream_results_rank.head()


,Imputation_Method,Task,Missing Type,Missing Fraction,Column,result_type,metric,Baseline,Corrupted,Imputed,...,name,MajorityClassSize,MinorityClassSize,NumberOfFeatures,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures,NumberOfClasses,Downstream Performance Rank,Data_Constellation
0,VAE,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.721942,0.721804,...,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,5.0,MAR - 0.01
1,Discriminative DL,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.721631,0.722056,...,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,4.0,MAR - 0.01
2,Random Forest,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.721506,0.722144,...,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,3.0,MAR - 0.01
3,Mean/Mode,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.720902,0.721964,...,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,2.0,MAR - 0.01
4,KNN,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.720294,0.722247,...,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,1.0,MAR - 0.01


## Analyzing Performance based on Rank and Improvement per Data Constellation

Hier die Rechnung -> Bestes Ergebnis pro "Experimental condition" - Beste Methode im Durchschnitt 

ToDo´s für restliche Auswertung (Mathematische Part)
- Beste Imp-Methode je Datensatz ermitteln (-> via Ranking am besten, je Konstellation (Bsp. MAR 0.01)
- durchschnittliche Platzierung jeder Imp-Methode ermitteln (Ranking -> dann je Konstellation (Bsp. MAR 0.01)
- Beste Imp je Datensatz mit durchschnittlich bester Imp vergleichen (Liste mit beste Imp & Liste mit Durchschn. Imp -> VGL)
(jede Konstellation genau einmal in jeder Liste)


In [153]:
data = downstream_results_rank

# Count amount of different Data constellations in column "Data_Constellation"
dc_unique = data.Data_Constellation.unique().size
print(dc_unique, "Data Constellations")
print("_____________________")
# Count amount of 1.0 Ranking result in column "Downstream Performance Rank" (Numbers must match)
rank_count = data['Downstream Performance Rank'].value_counts()
print(rank_count)
print("_____________________")
# Filter for 1.0 Ranking -> Overview -> save as csv
rank_1 = data.loc[data['Downstream Performance Rank'] == 1.0]
rank_1.to_csv('rank_1.csv')

print("_____________________")
# Count how often each Imputation Method is present -> most "wins"
rank_wins = rank_1['Imputation_Method'].value_counts()
print(rank_wins)
print("_____________________")
# Take initial overview and filter for each imputation method and calculate average rank
methods = ['Random Forest', 'KNN', 'Mean/Mode', 'VAE', 'GAIN', 'Discriminative DL']
for i in methods:
    df_average_rank = data.loc[data['Imputation_Method'] == i]
    len_ar = len(df_average_rank)
    print(len_ar, "Amount of results available")
    rank_pos = df_average_rank['Downstream Performance Rank'].value_counts().sort_index(ascending=True)
    print(rank_pos)
    average_rank = df_average_rank["Downstream Performance Rank"].mean()
    print("Average Rank for", i, "is", average_rank)
    average_improvement = df_average_rank["Improvement"].mean()
    print("Average Improvement to baseline is", average_improvement)
    print("_____________________")



12 Data Constellations
_____________________
1.0    245
2.0    206
5.0    193
4.0    193
3.0    189
6.0    152
Name: Downstream Performance Rank, dtype: int64
_____________________
_____________________
Discriminative DL    58
VAE                  50
KNN                  45
Mean/Mode            43
Random Forest        31
GAIN                 18
Name: Imputation_Method, dtype: int64
_____________________
204 Amount of results available
1.0    31
2.0    35
3.0    46
4.0    22
5.0    45
6.0    25
Name: Downstream Performance Rank, dtype: int64
Average Rank for Random Forest is 3.4411764705882355
Average Improvement to baseline is 0.027681932282311274
_____________________
204 Amount of results available
1.0    45
2.0    38
3.0    44
4.0    31
5.0    31
6.0    15
Name: Downstream Performance Rank, dtype: int64
Average Rank for KNN is 3.049019607843137
Average Improvement to baseline is 0.03084613290442985
_____________________
204 Amount of results available
1.0    43
2.0    40
3.0    20
4

In [154]:
rank_1.head()

,Imputation_Method,Task,Missing Type,Missing Fraction,Column,result_type,metric,Baseline,Corrupted,Imputed,...,name,MajorityClassSize,MinorityClassSize,NumberOfFeatures,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures,NumberOfClasses,Downstream Performance Rank,Data_Constellation
4,KNN,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.720294,0.722247,...,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,1.0,MAR - 0.01
9,Discriminative DL,6,MAR,0.1,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.709453,0.722138,...,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,1.0,MAR - 0.1
14,KNN,6,MAR,0.3,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.687250,0.723030,...,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,1.0,MAR - 0.3
19,KNN,6,MAR,0.5,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.661620,0.724723,...,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,1.0,MAR - 0.5
24,Random Forest,6,MCAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.720962,0.722385,...,letter,813.0,734.0,17.0,20000.0,16.0,1.0,26.0,1.0,MCAR - 0.01


In [155]:
# Take initial overview and filter best average imputation method and take filtered dataframe from 1.0 Ranking
# Where Data_Constellation identical -> Ranking 1.0 [Improvement] - Best_Imp_Method [Improvement]
# Write Difference in seperat column - > Calculate Average improvement

# Adjust the following depending on the previous results
av_best = data.loc[data['Imputation_Method'] == 'Discriminative DL']
av_best['Task'] = av_best['Task'].astype(str)
av_best['Data_Constellation'] = av_best['Data_Constellation'] + ' - ' + av_best['Task']

av_best = av_best[['Imputation_Method', 'Improvement', 'Data_Constellation', 'Downstream Performance Rank']]
av_best = av_best.rename(columns={'Imputation_Method':'Imputation_Method_average', 
                               'Improvement':'Improvement_average',
                                 'Downstream Performance Rank':'Downstream Performance Rank Average'})

#av_best.head()

rank_1['Task'] = rank_1['Task'].astype(str)
rank_1['Data_Constellation'] = rank_1['Data_Constellation'] + ' - ' + rank_1['Task']
rank_1 = rank_1[['Imputation_Method', 'Improvement', 'Data_Constellation', 'Downstream Performance Rank']]
rank_1 = rank_1.rename(columns={'Imputation_Method':'Imputation_Method_best', 
                               'Improvement':'Improvement_best',
                               'Downstream Performance Rank':'Downstream Performance Rank Best'})

performance_difference = pd.merge(av_best, rank_1, on='Data_Constellation')
performance_difference.head()

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\675066912.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\675066912.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\675066912.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

,Imputation_Method_average,Improvement_average,Data_Constellation,Downstream Performance Rank Average,Imputation_Method_best,Improvement_best,Downstream Performance Rank Best
0,Discriminative DL,0.000588,MAR - 0.01 - 6,4.0,KNN,0.002704,1.0
1,Discriminative DL,0.017566,MAR - 0.1 - 6,1.0,Discriminative DL,0.017566,1.0
2,Discriminative DL,0.037695,MAR - 0.3 - 6,5.0,KNN,0.049547,1.0
3,Discriminative DL,0.084636,MAR - 0.5 - 6,2.0,KNN,0.087385,1.0
4,Discriminative DL,0.001326,MCAR - 0.01 - 6,4.0,Random Forest,0.001971,1.0


In [156]:
performance_difference['Improvement_best'] = performance_difference['Improvement_best'] + 1
performance_difference['Improvement_average'] = performance_difference['Improvement_average'] + 1

performance_difference['Performance Difference Best to Average'] = performance_difference['Improvement_best'] - performance_difference['Improvement_average']
Average_Difference = performance_difference['Performance Difference Best to Average'].mean()
print("Average Difference", Average_Difference)


Average Difference 0.011516673331212545


In [157]:

performance_difference.to_csv('performance_difference.csv')

## Analysis and Ranking based on F1 Score

In [158]:
# Relative Difference in Percent -> Best Method to Average Best Method

AVERAGE_BEST_IMPUTATION_METHOD = "Discriminative DL"

data = downstream_results_rank
data['Task'] = data['Task'].astype(str)
data['Data_Constellation_full'] = data['Data_Constellation'] + ' - ' + data['Task']

# TODO: drop unnecessary columns here
dc_unique = data.Data_Constellation_full.unique()
#print(dc_unique)

#data_constellations = ['MAR - 0.01', 'MAR - 0.1', 'MAR - 0.3', 'MCAR - 0.5', 'MCAR - 0.01', 'MCAR - 0.1', 'MCAR - 0.3', 'MCAR - 0.5', 'MNAR - 0.01', 'MNAR - 0.1', 'MNAR - 0.3', 'MNAR - 0.5']
data_constellations = dc_unique.tolist()
methods = ['Random Forest', 'KNN', 'Mean/Mode', 'VAE', 'GAIN', 'Discriminative DL']
#print(data_constellations)
#print(type(methods))
average_best_complete = pd.DataFrame()


for i in data_constellations:
    data_constel = data.loc[data['Data_Constellation_full'] == i]
    best_score = data_constel.loc[data_constel['Downstream Performance Rank'] == 1.0]
    average_best = data_constel.loc[data_constel['Imputation_Method'] == AVERAGE_BEST_IMPUTATION_METHOD]
    if(average_best.empty == True):
        print('empty')
    else:
        best_score_int = best_score.iloc[0]['Imputed']
    #print(best_score_int)
        average_best_int = average_best.iloc[0]['Imputed']
    #print(average_best_int)
        calc_result = ((best_score_int - average_best_int)/average_best_int)
#    print(calc_result)
#    print(i)
        average_best['Performance Difference to Best to Average in Percent'] = calc_result
        average_best_complete = average_best_complete.append(average_best)

average_best_complete

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\4166178819.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\4166178819.py:36: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\4166178819.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\4166178819.py:36: FutureWarning

empty


C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\4166178819.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\4166178819.py:36: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\4166178819.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\4166178819.py:36: FutureWarning

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\4166178819.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\4166178819.py:36: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\4166178819.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\4166178819.py:36: FutureWarning

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\4166178819.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\4166178819.py:36: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\4166178819.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\4166178819.py:36: FutureWarning

,Imputation_Method,Task,Missing Type,Missing Fraction,Column,result_type,metric,Baseline,Corrupted,Imputed,...,MinorityClassSize,NumberOfFeatures,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures,NumberOfClasses,Downstream Performance Rank,Data_Constellation,Data_Constellation_full,Performance Difference to Best to Average in Percent
1,Discriminative DL,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.721631,0.722056,...,734.0,17.0,20000.0,16.0,1.0,26.0,4.0,MAR - 0.01,MAR - 0.01 - 6,0.000265
9,Discriminative DL,6,MAR,0.1,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.709453,0.722138,...,734.0,17.0,20000.0,16.0,1.0,26.0,1.0,MAR - 0.1,MAR - 0.1 - 6,0.000000
10,Discriminative DL,6,MAR,0.3,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.694927,0.722148,...,734.0,17.0,20000.0,16.0,1.0,26.0,5.0,MAR - 0.3,MAR - 0.3 - 6,0.001221
18,Discriminative DL,6,MAR,0.5,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.660129,0.721248,...,734.0,17.0,20000.0,16.0,1.0,26.0,2.0,MAR - 0.5,MAR - 0.5 - 6,0.004819
21,Discriminative DL,6,MCAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.721357,0.722314,...,734.0,17.0,20000.0,16.0,1.0,26.0,4.0,MCAR - 0.01,MCAR - 0.01 - 6,0.000099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1157,Discriminative DL,41671,MCAR,0.5,a9,downstream_performance_mean,Classification Tasks,0.241342,0.251922,0.241354,...,743.0,21.0,20000.0,20.0,1.0,5.0,1.0,MCAR - 0.5,MCAR - 0.5 - 41671,0.000000
1161,Discriminative DL,41671,MNAR,0.01,a9,downstream_performance_mean,Classification Tasks,0.241342,0.241122,0.241342,...,743.0,21.0,20000.0,20.0,1.0,5.0,2.0,MNAR - 0.01,MNAR - 0.01 - 41671,-0.085789
1167,Discriminative DL,41671,MNAR,0.1,a9,downstream_performance_mean,Classification Tasks,0.241342,0.244979,0.241013,...,743.0,21.0,20000.0,20.0,1.0,5.0,1.0,MNAR - 0.1,MNAR - 0.1 - 41671,0.000000
1172,Discriminative DL,41671,MNAR,0.3,a9,downstream_performance_mean,Classification Tasks,0.241342,0.251333,0.241342,...,743.0,21.0,20000.0,20.0,1.0,5.0,1.0,MNAR - 0.3,MNAR - 0.3 - 41671,0.000000


In [159]:
average_difference = average_best_complete['Performance Difference to Best to Average in Percent'].mean()
print(average_difference, "average difference in Percent")

0.010088907885811365 average difference in Percent


In [160]:
# Relative Difference in absolute values (F1 Score) -> Best Method to Average Best Method

AVERAGE_BEST_IMPUTATION_METHOD = "Discriminative DL"

data = downstream_results_rank
data['Task'] = data['Task'].astype(str)
data['Data_Constellation_full'] = data['Data_Constellation'] + ' - ' + data['Task']

# TODO: drop unnecessary columns here
dc_unique = data.Data_Constellation_full.unique()
#print(dc_unique)

#data_constellations = ['MAR - 0.01', 'MAR - 0.1', 'MAR - 0.3', 'MCAR - 0.5', 'MCAR - 0.01', 'MCAR - 0.1', 'MCAR - 0.3', 'MCAR - 0.5', 'MNAR - 0.01', 'MNAR - 0.1', 'MNAR - 0.3', 'MNAR - 0.5']
data_constellations = dc_unique.tolist()
methods = ['Random Forest', 'KNN', 'Mean/Mode', 'VAE', 'GAIN', 'Discriminative DL']
#print(data_constellations)
#print(type(methods))
average_best_total = pd.DataFrame()


for i in data_constellations:
    data_constel = data.loc[data['Data_Constellation_full'] == i]
    best_score = data_constel.loc[data_constel['Downstream Performance Rank'] == 1.0]
    average_best = data_constel.loc[data_constel['Imputation_Method'] == AVERAGE_BEST_IMPUTATION_METHOD]
    if(average_best.empty == True):
        print('empty')
    else:
        best_score_int = best_score.iloc[0]['Imputed']
    #print(best_score_int)
        average_best_int = average_best.iloc[0]['Imputed']
    #print(average_best_int)
        calc_result = (best_score_int - average_best_int)
#    print(calc_result)
#    print(i)
        average_best['Performance Difference to Best to Average in absolute'] = calc_result
        average_best_total = average_best_total.append(average_best)
 
average_best_total


C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\2539710273.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\2539710273.py:36: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\2539710273.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\2539710273.py:36: FutureWarning

empty


C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\2539710273.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\2539710273.py:36: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\2539710273.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\2539710273.py:36: FutureWarning

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\2539710273.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\2539710273.py:36: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\2539710273.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\2539710273.py:36: FutureWarning

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\2539710273.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\2539710273.py:36: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\2539710273.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\dittr\AppData\Local\Temp\ipykernel_21424\2539710273.py:36: FutureWarning

,Imputation_Method,Task,Missing Type,Missing Fraction,Column,result_type,metric,Baseline,Corrupted,Imputed,...,MinorityClassSize,NumberOfFeatures,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures,NumberOfClasses,Downstream Performance Rank,Data_Constellation,Data_Constellation_full,Performance Difference to Best to Average in absolute
1,Discriminative DL,6,MAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.721631,0.722056,...,734.0,17.0,20000.0,16.0,1.0,26.0,4.0,MAR - 0.01,MAR - 0.01 - 6,0.000191
9,Discriminative DL,6,MAR,0.1,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.709453,0.722138,...,734.0,17.0,20000.0,16.0,1.0,26.0,1.0,MAR - 0.1,MAR - 0.1 - 6,0.000000
10,Discriminative DL,6,MAR,0.3,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.694927,0.722148,...,734.0,17.0,20000.0,16.0,1.0,26.0,5.0,MAR - 0.3,MAR - 0.3 - 6,0.000882
18,Discriminative DL,6,MAR,0.5,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.660129,0.721248,...,734.0,17.0,20000.0,16.0,1.0,26.0,2.0,MAR - 0.5,MAR - 0.5 - 6,0.003476
21,Discriminative DL,6,MCAR,0.01,x-box,downstream_performance_mean,Classification Tasks,0.722137,0.721357,0.722314,...,734.0,17.0,20000.0,16.0,1.0,26.0,4.0,MCAR - 0.01,MCAR - 0.01 - 6,0.000071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1157,Discriminative DL,41671,MCAR,0.5,a9,downstream_performance_mean,Classification Tasks,0.241342,0.251922,0.241354,...,743.0,21.0,20000.0,20.0,1.0,5.0,1.0,MCAR - 0.5,MCAR - 0.5 - 41671,0.000000
1161,Discriminative DL,41671,MNAR,0.01,a9,downstream_performance_mean,Classification Tasks,0.241342,0.241122,0.241342,...,743.0,21.0,20000.0,20.0,1.0,5.0,2.0,MNAR - 0.01,MNAR - 0.01 - 41671,-0.020704
1167,Discriminative DL,41671,MNAR,0.1,a9,downstream_performance_mean,Classification Tasks,0.241342,0.244979,0.241013,...,743.0,21.0,20000.0,20.0,1.0,5.0,1.0,MNAR - 0.1,MNAR - 0.1 - 41671,0.000000
1172,Discriminative DL,41671,MNAR,0.3,a9,downstream_performance_mean,Classification Tasks,0.241342,0.251333,0.241342,...,743.0,21.0,20000.0,20.0,1.0,5.0,1.0,MNAR - 0.3,MNAR - 0.3 - 41671,0.000000


In [161]:
average_difference = average_best_total['Performance Difference to Best to Average in absolute'].mean()
print(average_difference, "average difference in absolut")

0.0024169639950043483 average difference in absolut


## Heatmap (needs to be adjusted)

In [162]:
df_heat = pd.read_csv('downstream_results_rank_temp.csv')
#df_heat = downstream_results_rank
df_heat.drop(["Missing Type", "Missing Fraction", "Column", "result_type", "metric", "Baseline", "Imputed", "Corrupted", "Unnamed: 0", "Unnamed: 0", "name", "NumberOfClasses", "MajorityClassSize", "MinorityClassSize"], axis=1, inplace=True)
#df_heat['Improvement'] = df_heat['Improvement'] -1
df_heat

,Unnamed: 0.1,Imputation_Method,Task,Improvement,NumberOfFeatures,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures,Downstream Performance Rank,Data_Constellation
0,0,VAE,6,-0.000192,17.0,20000.0,16.0,1.0,5.0,MAR - 0.01
1,1,Discriminative DL,6,0.000588,17.0,20000.0,16.0,1.0,4.0,MAR - 0.01
2,2,Random Forest,6,0.000884,17.0,20000.0,16.0,1.0,3.0,MAR - 0.01
3,3,Mean/Mode,6,0.001471,17.0,20000.0,16.0,1.0,2.0,MAR - 0.01
4,4,KNN,6,0.002704,17.0,20000.0,16.0,1.0,1.0,MAR - 0.01
...,...,...,...,...,...,...,...,...,...,...
1173,1173,KNN,41671,-0.094887,21.0,20000.0,20.0,1.0,5.0,MNAR - 0.5
1174,1174,Random Forest,41671,-0.093526,21.0,20000.0,20.0,1.0,4.0,MNAR - 0.5
1175,1175,VAE,41671,-0.091875,21.0,20000.0,20.0,1.0,3.0,MNAR - 0.5
1176,1176,Mean/Mode,41671,-0.088478,21.0,20000.0,20.0,1.0,2.0,MNAR - 0.5


In [163]:
# Get a dataframe for each "Data_Constellation"
# Hier mit Variablen arbeiten -> Liste mit Konstellationen

# Hier eventuell for schleife, etc


# drop unneccessary columns

#df_heat = downstream_results_rank
#df_heat.drop(["Missing Type", "Missing Fraction", "Column", "result_type", "metric", "Baseline", "Imputed", "Corrupted", "Unnamed: 0", "Unnamed: 0", "name", "NumberOfClasses", "MajorityClassSize", "MinorityClassSize"], axis=1, inplace=True)

#df_heat['Improvement'] = df_heat['Improvement']
df_heat = df_heat.astype({"Task":"string"})

#mar001.drop(["Missing Type", "Missing Fraction", "Column", "result_type", "metric", "Baseline", "Imputed", "Corrupted", "Unnamed: 0"], axis=1, inplace=True)

data_constellations = ['MAR - 0.01', 'MAR - 0.1', 'MAR - 0.3', 'MCAR - 0.5', 'MCAR - 0.01', 'MCAR - 0.1', 'MCAR - 0.3', 'MCAR - 0.5', 'MNAR - 0.01', 'MNAR - 0.1', 'MNAR - 0.3', 'MNAR - 0.5']


for i in data_constellations:
    data_constel = df_heat.loc[df_heat['Data_Constellation'] == i]

    ### uncomment whatever you want to investigate

    ## sort by amount datapoints (ascending)
    data_constel = data_constel.sort_values(by=['NumberOfInstances'])

    ## sort by amount of features (ascending)
    #data_constel = data_constel.sort_values(by=['NumberOfFeatures'])

    ## sort by amount of datapoints and features (ascending)
    #data_constel = data_constel.sort_values(by=['NumberOfInstances', 'NumberOfFeatures'])

    ## sort by amount of categorical features and datapoints (ascending)
    #data_constel = data_constel.sort_values(by=['NumberOfCategoricalFeatures', 'NumberOfInstances'])

    ## sort by amount of numerical features and datapoints (ascending)
    #data_constel = data_constel.sort_values(by=['NumberOfNumericFeatures', 'NumberOfInstances'])
    
    Dataset_number = data_constel["Task"]
    Imputation_Method = data_constel["Imputation_Method"]
    Improvement = data_constel["Improvement"]
    

    trace = go.Heatmap(
                   z=Improvement,
                   x=Dataset_number,
                   y=Imputation_Method,
                   type = 'heatmap',
                    autocolorscale= False,
                    colorscale = 'RdBu_r',
                    zmid=0,
                    #hoverinfo='text',
                    #text=hovertext
                    )
    data = [trace]
    fig = go.Figure(data=data)
    fig.update_layout(
        title=i,
        xaxis_nticks=36)
    fig.show()



In [164]:
# Seaborn heatmap
'''
plt.subplots(figsize=(50,10))
sns.set()

heatmap_mar001 = mar001.pivot("Imputation_Method", "Task", "Improvement")
ax = sns.heatmap(heatmap_mar001, annot=True, vmin=-0.1, vmax=0.2)
#ax = sns.heatmap(mar001, annot=True, vmin=-0.3, vmax=0.3)
#plt.subplots(figsize=(50,50))
title = mar001.iloc[2]['Data_Constellation']
print(title)
plt.title(title)
plt.show()
'''

'\nplt.subplots(figsize=(50,10))\nsns.set()\n\nheatmap_mar001 = mar001.pivot("Imputation_Method", "Task", "Improvement")\nax = sns.heatmap(heatmap_mar001, annot=True, vmin=-0.1, vmax=0.2)\n#ax = sns.heatmap(mar001, annot=True, vmin=-0.3, vmax=0.3)\n#plt.subplots(figsize=(50,50))\ntitle = mar001.iloc[2][\'Data_Constellation\']\nprint(title)\nplt.title(title)\nplt.show()\n'

## Plotly Heatmaps Tests

In [165]:
#heatmap_mar001.head()


In [166]:
#testmar001 = xr.tutorial.open_dataset('air_temperature').air.sel(lon=250.0)

'''
#plotly express test

fig = px.imshow(heatmap_mar001, text_auto = True, 
                labels=dict(x="Task", y="Imputation_Method", color="Improvement"),
                color_continuous_scale='RdBu_r', color_continuous_midpoint=0)
'''

'\n#plotly express test\n\nfig = px.imshow(heatmap_mar001, text_auto = True, \n                labels=dict(x="Task", y="Imputation_Method", color="Improvement"),\n                color_continuous_scale=\'RdBu_r\', color_continuous_midpoint=0)\n'

In [167]:
### uncomment whatever you want to investigate

## sort by amount datapoints (ascending)
#mar001 = mar001.sort_values(by=['NumberOfInstances'])

## sort by amount of features (ascending)
#mar001 = mar001.sort_values(by=['NumberOfFeatures'])

## sort by amount of datapoints and features (ascending)
#mar001 = mar001.sort_values(by=['NumberOfInstances', 'NumberOfFeatures'])

## sort by amount of categorical features and datapoints (ascending)
#mar001 = mar001.sort_values(by=['NumberOfCategoricalFeatures', 'NumberOfInstances'])

## sort by amount of numerical features and datapoints (ascending)
#mar001 = mar001.sort_values(by=['NumberOfNumericFeatures', 'NumberOfInstances'])



mar001 = mar001.astype({"Task":"string"})

Dataset_number = mar001["Task"]
Imputation_Method = mar001["Imputation_Method"]
Improvement = mar001["Improvement"]




NameError: name 'mar001' is not defined

In [ ]:
trace = go.Heatmap(
                   z=Improvement,
                   x=Dataset_number,
                   y=Imputation_Method,
                   type = 'heatmap',
                    autocolorscale= False,
                    colorscale = 'RdBu_r',
                    zmid=0,
                    hoverinfo='text',
                    text=hovertext
                    )




data = [trace]
fig = go.Figure(data=data)
#iplot(fig)


fig.show()

ToDo´s für Darstellung:
- Optionen für einfache Anpassung bei der Sortierung/Darstellung:
    - Anzahl Datenpunkte
    - Anzahl Features
    - Anzahl numerische Features
    - Anzahl kategorische Features
- Schleife aufsetzen für alle Datenkonstellationen (nicht hart kodieren)
- Jeweils beste Imputationsmethode je Datensatz nochmals separat in Heatmap



ToDo´s für restliche Auswertung (Mathematische Part)
- Beste Imp-Methode je Datensatz ermitteln (-> via Ranking am besten, je Konstellation (Bsp. MAR 0.01)
- durchschnittliche Platzierung jeder Imp-Methode ermitteln (Ranking -> dann je Konstellation (Bsp. MAR 0.01)
- Beste Imp je Datensatz mit durchschnittlich bester Imp vergleichen (Liste mit beste Imp & Liste mit Durchschn. Imp -> VGL)
(jede Konstellation genau einmal in jeder Liste)



Sonstiges (keine Prio)
- Optionen für Filterung (bei Beadrf umsetzen -> vorerst keine Priorität!)
    - Numerisches Feature wurde imputiert
    - Kategorisches Feature wurde imputiert

## Application Scenario 2 - Downstream Performance

### Categorical  Columns (Classification)

In [ ]:
'''
draw_cat_box_plot(
    downstream_results,
    "Improvement",
    (-0.15, 0.3),
    FIGURES_PATH,
    "fully_observed_downstream_boxplot.eps",
    hue_order=list(rename_imputer_dict.values()),
    row_order=list(rename_metric_dict.values())
)
'''
# Not used at the moment -> function from other file required, check first field